# Plot Geo-Points Map with Folium
# ----------------------------------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
import requests
from xml.etree import ElementTree
import folium

GetAllCountersUrl = "http://webservices.commuterpage.com/counters.cfc?wsdl&method=GetAllCounters"

xmlfile = open('xml_getallcounters.xml', 'w') 	# create & open writable .xml file
xmldata = requests.get(GetAllCountersUrl)		# get data
xmlfile.write(xmldata.text)						# write data in the file
xmlfile.close()									# close file
xml_data = 'xml_getallcounters.xml'				# name path file

In [6]:
# Parse the xml data. 
tree = ElementTree.parse(xml_data)

In [7]:
counter = tree.find('counter')
name = counter.find('name')
name.text

'110 Trail'

In [10]:
lat = counter.find('latitude')
lon = counter.find('longitude')
region = counter.find('region/name')

counter.text, name.text, lat.text, lon.text, region.text

('\n\t\t\t', '110 Trail', '38.885315', '-77.065022', 'Arlington')

In [11]:
# COVERT XML TO DATAFRAME:
# Create DataFrame looping .xml tree and adding an ID numbe.
id = []
name = []
latitude = []
longitude = []
region = []


for c in tree.findall('counter'):
    id.append(c.attrib['id'])
    name.append(c.find('name').text)
    latitude.append(c.find('latitude').text)
    longitude.append(c.find('longitude').text)
    region.append(c.find('region/name').text)

df_counters = pd.DataFrame(
    {'ID' : id,
     'Name' : name,
     'latitude' : latitude,
     'longitude' : longitude,
     'region' : region
    })
df_counters.head()

,ID,Name,latitude,longitude,region
0,33,110 Trail,38.885315,-77.065022,Arlington
1,30,14th Street Bridge,38.874260,-77.044610,Arlington
2,43,15th Street NW,38.907470,-77.034610,DC
3,32,Arlington Mill Trail,38.845610,-77.096046,Arlington
4,24,Ballston Connector,38.882950,-77.121235,Arlington


In [16]:
locations = df_counters[['latitude', 'longitude']]
locationlist = locations.values.tolist()
len(locationlist), locationlist[7]

(51, ['38.979500', '-77.096760'])

In [18]:
# PLOT MAP & LOCATIONS:
map = folium.Map(location=[38.9, -77.05], zoom_start=12)
map

In [22]:
# Add geo data points.
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(map)
map

In [26]:
# ADD STYLE TO PLOT:
from folium.plugins import MarkerCluster

map2 = folium.Map(location=[38.9, -77.05], tiles='CartoDB dark_matter', zoom_start=11)
# Clusterig marker
marker_cluster = folium.plugins.MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(marker_cluster)
map2

In [28]:
# add a bicycle icon.The bicycle icon was not available in version 0.2.1 of Folium, so I instead went with a pedestrian looking icon.

map2 = folium.Map(location=[38.9, -77.05], tiles='Stamen Terrain', zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point], icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map2


In [29]:
# assign a unique color to each region.
def regioncolors(counter):
    if counter['region'] == 'Arlington':
        return 'green'
    elif counter['region'] == 'Alexandria':
        return 'blue'
    elif counter['region'] == 'DC':
        return 'red'
    else:
        return 'darkblue'
df_counters["color"] = df_counters.apply(regioncolors, axis=1)
df_counters.head()

,ID,Name,latitude,longitude,region,color
0,33,110 Trail,38.885315,-77.065022,Arlington,green
1,30,14th Street Bridge,38.874260,-77.044610,Arlington,green
2,43,15th Street NW,38.907470,-77.034610,DC,red
3,32,Arlington Mill Trail,38.845610,-77.096046,Arlington,green
4,24,Ballston Connector,38.882950,-77.121235,Arlington,green


In [31]:
map3 = folium.Map(location=[38.9, -77.05], tiles='CartoDB positron', zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='ID:'+df_counters['ID'][point]+' '+df_counters['Name'][point], icon=folium.Icon(color=df_counters["color"][point], icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map3